In [2]:
from utils import kappa, ode_system, dl_func, ratio, fs8, chi_squared

In [3]:
import numpy as np

import emcee

from scipy.optimize import minimize
from scipy.integrate import solve_ivp

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

In [4]:
om_fdl = [0.3, 0.3, 0.266, 0.3, 0.31, 0.3, 0.27, 0.27, 0.25, 0.25,
          0.274,0.307115, 0.27, 0.27, 0.27, 0.3, 0.3, 0.27]

file_path = 'data/Cij_WiggleZ.txt'
Wiggle = np.loadtxt(file_path)

data_file_path = 'data/data_growth_2016_main.txt'
data = np.loadtxt(data_file_path)

In [5]:
z_data = data[:,0]
y_data = data[:,1]
sigma = data[:,2]

cov = np.eye(len(z_data))

# Replace the diagonal of the identity matrix with the array elements
np.fill_diagonal(cov, sigma**2)
cov[12:15, 12:15] = Wiggle
cov_inv = np.linalg.inv(cov)

a_data = 1/(z_data+1)

a_data = a_data[::-1]
y_data = y_data[::-1]
sigma  = sigma[::-1]
om_fdl = om_fdl[::-1]

In [7]:

ai,af,da = 1e-3,1, 1e-4
yi,xi = ai,1
N = int(round((af-ai)/da))
a_values = np.linspace(ai, af, N)
indices = np.searchsorted(a_values, a_data, side="left")
a_values[indices] = a_data
initial_conditions = [yi, xi]

initial_guess = [ 0.3, 0.8]
nll = lambda *args: chi_squared(*args)
result = minimize(nll, initial_guess, args = (fs8, a_values, a_data, y_data, cov_inv, om_fdl), method='Nelder-Mead')

optimal_params = result.x

print(result.x, result.message, result.success)

[0.24172271 0.85914095] Optimization terminated successfully. True


In [8]:
def lnprior(theta):
  b1, sig = theta
  ones = np.ones(N)
  if -2< b1 < 2  and 0<= sig <= 2 and (kappa(a_values,0,b1)>=ones).all():
    return 0.
  return -np.inf

def lnprob(theta, model, a_values, a_data, y_data, cov_inv, om_fdl):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp - chi_squared(theta, model, a_values, a_data, y_data, cov_inv, om_fdl)

runs = 3000

nwalkers = 30
pos = [result.x + 1e-3*np.random.randn(2) for i in range(nwalkers)]

sampler = emcee.EnsembleSampler(nwalkers, 2, lnprob, args=(fs8, a_values, a_data, y_data, cov_inv, om_fdl))

sampler.run_mcmc(pos, runs, progress=True)

100%|██████████| 3000/3000 [22:03<00:00,  2.27it/s]


State([[0.29345584 0.97178587]
 [0.28499027 0.97276635]
 [0.19146093 0.78001448]
 [0.27026788 0.90466139]
 [0.32150966 1.02167903]
 [0.26648317 0.90612705]
 [0.26911047 0.9132883 ]
 [0.30658221 1.00071446]
 [0.30608294 1.02431558]
 [0.21341272 0.81653202]
 [0.26383247 0.84926104]
 [0.22553623 0.86493191]
 [0.24997321 0.85646837]
 [0.18121033 0.74253451]
 [0.17758509 0.7735723 ]
 [0.27539436 0.95726211]
 [0.2666     0.87615656]
 [0.19500181 0.74395909]
 [0.1944967  0.76919397]
 [0.19746106 0.79439019]
 [0.24105039 0.83647562]
 [0.27845094 0.90150025]
 [0.2977409  1.00287876]
 [0.20310625 0.74173136]
 [0.25718778 0.87803317]
 [0.29263422 0.97881327]
 [0.15001423 0.70903902]
 [0.19116558 0.81156816]
 [0.20570887 0.76801269]
 [0.20247748 0.78355718]], log_prob=[-14.62137145 -14.84285383 -14.58937889 -14.39654593 -16.00563166
 -14.24314948 -14.26184375 -14.96942942 -15.19203676 -14.32485336
 -16.68835623 -15.43503779 -14.45869728 -14.9476022  -15.41074657
 -15.00369294 -15.14582371 -15.8276

In [9]:
samples = sampler.chain[:].reshape((-1, 2))
np.savez('samples_GCDM.npz', samples)